Groundwater | Case Study

# Topic 1 : Introduction to the Groundwater course - The Limmat Vally Aquifer 

Dr. Xiang-Zhao Kong & Dr. Beatrice Marti & Louise Noel du Prat

In [ ]:
# Setting up the notebook
import sys
import os

from IPython.display import display
import rasterio
import plotly.graph_objects as go

# Load local modules
sys.path.append('../')
sys.path.append('../SUPPORT_REPO/src')
sys.path.append('../SUPPORT_REPO/src/scripts/scripts_exercises')
import climate_utils as cu
import print_images as du
import river_utils as ru
from data_utils import download_named_file
from map_utils import display_groundwater_resources_map
from progress_tracking import (
    create_perceptual_model_progress_tracker,
    create_section_completion_marker,
)

# Introducing the Case Study & Building a Perceptual Model
Most exercises as well as the final project in this course are based on case studies. Case studies are real-world scenarios that provide you with an opportunity to apply your knowledge and skills to solve complex groundwater problems. In this first year we choose the Limmat valley aquifer in Zurich, Switzerland. 

We will follow the actual steps a professional groundwater modeler might take to set up a numerical model to answer a specific question. In a first step, the modelers familiarize themselves with the Limmat valley aquifer and its hydrological and hydrogeological properties. We start out with a very generic perceptual model of the aquifer (see Figure 1), which we'll refine as we uncover new information. 

In [ ]:
du.display_image(
    image_filename='perceptual_model_00_initial.png', 
    image_folder='1_perceptual_model', 
    caption='Figure 1: Initial perceptual model which will be refined and used as a basis for a first long-term average water balance estimation.'
)

We first try to get an understanding of the hydrogeology of the Limmat valley aquifer. And then go through each water balance component and try to get a first rough understanding of the most important fluxes in the system. At a later stage, we will re-use and refine the perceptual model to set up a numerical model of the Limmat valley aquifer.

# The Limmat Valley Aquifer
The Limmat valley aquifer is a well-studied groundwater reservoir beneath the city of Zurich, Switzerland. Doppler and colleagues write, that it was formed during the last ice age, when the Lindt glacier retreated. The Limmat valley aquifer has no direct hydraulic connection to lake Zurich in the east where it is bound by impermeable lake sediments and moraine material. The aquifer is further confined in the north and south by the side morains of the Lindth glacier. Lateral inflow of groundwater from the hills in the north and south is to be expected. The groundwater body is further in connection with the river Sihl in the east and the river Limmat in the north. The hydraulic properties of the aquifer are higly heterogenic because of its complex geological history formed through various sedimentation and erosion events from the rivers Sihl and Limmat. [\[1, 2\]](#references)

In Figure 2 you see a prinscreen of the [GIS-browser](https://www.gis.zh.ch/) of the canton of Zurich. The cyanide blue area in the center of the map shows the Limmat valley aquifer. The darker the color, the larger the thickness of the aquifer. The GIS-Browser is only available in German. You can use the translation feature of your browser to translate the page into your preferred language.

In [ ]:
du.display_image(
    image_filename='GIS-browser_canton_Zurich.png', 
    image_folder='1_perceptual_model',
    caption='Figure 2: Printscreen of the GIS-browser of the canton of Zurich displaying the cantonal groundwater map. Source: https://www.gis.zh.ch/, accessed 2024-05-01'
)

We now start working through the available information to refine our perceptual model of the Limmat valley aquifer. We will use the GIS-browser to refine our understanding of the geometry of the aquifer and how we want to represent it in the model. 

You can use the list below to track your progress through the perceptual model construction. 

In [ ]:
create_perceptual_model_progress_tracker()

# Aquifer Geometry
Most of the layers in the GIS-Browser of the Canton of Zurich are available to the general public for download (button "Datenbezug" in the upper right corner of the map pane). We use this feature and import the relevant parts of the map into our JupyterHub environment. 

## Bottom Topography
We have downloaded the aquifer thickness layer for you from the GIS-browser and clipped it to the area of interest (AOI) of the Limmat valley aquifer. Let's have a look at the bottom topography of the aquifer.

In [ ]:
# This function downloads the groundwater map of the canton of Zurich and saves 
# it to the folder applied_groundwater_modelling_data in your home directory. 
gw_map_path = download_named_file(
    name='groundwater_map_norm', 
    data_type='gis', 
)


In [ ]:
# Create and display the interactive map
interactive_map = display_groundwater_resources_map(gw_map_path, zoom_level=11)
interactive_map

We can use these contours of the aquifer thickness to build the bottom of the aquifer in our model later on.  

## Top Topography
The Limmat valley aquifer is unconfined. We can use the topography of the Limmat valley to represent the top of the aquifer. SwissTopo provides digital elevation models (DEMs) of Switzerland in various resolutions. We use the 25m resolution DEM for our case study. We have downloaded the DEM from the [SwissTopo website](https://www.swisstopo.admin.ch/en/geodata/height/dhm25.html) and clipped it to the area of interest (AOI) of the Limmat valley aquifer. If you are interested to learn more about the DEM and how to clip it to the AOI, you can read the [processing_DEM.ipynb](../SUPPORT_REPO/src/scripts/scripts_limmat_data_preprocessing/processing_DEM.ipynb) notebook.

In [ ]:
dem_path = download_named_file(
    name='dem',
    data_type='gis'
)

In [ ]:
# Open the DEM file
with rasterio.open(dem_path) as dem_src:
    dem_data = dem_src.read(1)  # Read the first band
    transform = dem_src.transform
    height, width = dem_src.height, dem_src.width
    x_coords = [transform.c + i * transform.a for i in range(width)]
    y_coords = [transform.f + i * transform.e for i in range(height)]

# Use the clipped and masked DEM from the previous steps
fig = go.Figure(data=go.Heatmap(
    z=dem_data,
    x=x_coords,
    y=y_coords,
    colorscale='earth',
    zmin=0,
    colorbar=dict(title='Elevation (m)'), 
))

fig.update_layout(
    title='Interactive DEM of the Limmat Valley (Clipped Area)',
    xaxis_title='Easting (m)',
    yaxis_title='Northing (m)',
    # Keep aspect ratio correct
    yaxis_scaleanchor="x",
    yaxis_scaleratio=1,
    xaxis_tickformat='d',
    yaxis_tickformat='d'
)

fig.show()

The topography of the Limmat valley is quite flat, with a maximum elevation of about 400 m above sea level. As we use a rather coarse DEM resolution of 25 m, we can not see the small topographic features in the Limmat valley, like for example the river beds. As we would like to model the river-aquifer interaction, we need to find a suitable compromise between DEM (and model grid) resolution and computational capacities to represent the river beds in our model.  

River-aquifer interaction is predominantly driven by the stream bed elevation relative to the groundwater table. Therefore, an accurate representation of the streambed elevation in the numerical groundwater flow model is crucial. With a spatial resolution of 20 m, the DEM is not sufficient to represent the river beds in the Limmat valley aquifer. We will discuss the river-aquifer interaction in a later chapter, where we will also discuss how to represent the river beds in our model for different modelling purposes in the case studies. 

## Lateral Model Boundaries
The groundwater map of the Limmat valley aquifer shows that the aquifer is not hydraulically connected to the lake Zurich in the east, but that it is bounded by impermeable lake sediments and moraine material. The aquifer is further confined in the north and south by the side morains of the Lindth glacier. Lateral inflow of groundwater from the hills in the north and south is to be expected. The most common approach is to use a water balance approach on the hill sides to estimate the sub-surface inflow from the hills. We'll address this in the chapter on [climate data](#climate). We do not consider the lateral inflow from the Sihl valley aquifer into the Limmat valley aquifer, as the inflow is rather small and parallel to the general flow direction in the Limmat valley. 

Should you have to estimate the lateral inflow from the Sihl valley aquifer, you can apply Darcy's law under the Dupuit assumption. In practice, the organization tasking you to produce a groundwater flow model will typically provide you with estimates of hydraulic conductivities from field measurements. As a first estimate, you can use the hydraulic conductivities from the literature (e.g. Freeze and Cherry, 1979)[\[4\]](#references).   

In [ ]:
du.display_image(
    image_filename='hydraulic_conductivities_by_Freeze.png', 
    image_folder='1_perceptual_model',
    caption='Hydraulic conductivity ranges as presented in the book <<Groundwater>> by Freeze and Cherry (1979). The ranges are based on the hydraulic conductivity of unconsolidated sediments. Hydraulic conductivity is typically known only in approximate orders of magnitude.'
)


> 📚 **Theory reminder**: Applying Darcy's Law in Unconfined Aquifers: The Dupuit Approximation
>
> To apply Darcy's Law for calculating the total discharge through an unconfined aquifer, we often rely on a set of simplifying assumptions known as the *Dupuit approximation* (or Dupuit-Forchheimer assumption). These assumptions allow us to treat the complex three-dimensional flow problem as a simpler two-dimensional one by integrating flow over the entire aquifer thickness.
> 
> The key conditions that must be met for the Dupuit approximation to be valid are:
> 
> *   *The water table gradient is small:* The slope of the phreatic surface must be gentle. This is the most critical assumption, as it implies that flow is almost entirely horizontal.
> *   *Flow is horizontal:* It is assumed that the hydraulic head is constant with depth for any given vertical line. This means equipotential lines are vertical, and consequently, the flow lines are horizontal.
> *   *The hydraulic gradient is equal to the slope of the water table:* The slope of the water table is assumed to be the hydraulic gradient (`i`) for the entire saturated thickness of the aquifer.
> 
> When these conditions hold, we can use a modified form of Darcy's Law to calculate the discharge `Q` of the aquifer:
> 
> `Q = -K * h * B * (dh/dx)`
> 
> Where:
> *   `Q` is the discharge (e.g., m3/s)
> *   `K` is the hydraulic conductivity (e.g., m/s)
> *   `h` is the saturated thickness (m)
> *   `B` is the width of the aquifer perpendicular to the flow direction (m)
> *   `dh/dx` is the slope of the water table, which represents the hydraulic gradient.
> 
> In essence, the Dupuit approximation simplifies reality by ignoring the vertical components of flow within the aquifer, which is a reasonable simplification when the water table is nearly flat compared to the saturated thickness.

For the outflow, a fixed-head boundary is typically chosen. This requires a known groundwater table at the boundary, e.g. a monitoring piezometer. Another option is to use the water level of a receiving stream or lake as a fixed-head boundary condition. The outflow boundary should be placed far enough away from the area of interest, so that the groundwater flow in the area of interest is not influenced by the boundary condition. For regional model, this is typically several hundred meters to several kilometers away from the area of interest. 

For our case, we choose the groundwater level at piezometer 481 as the fixed-head boundary condition. This relieves us of the need to implement the more complex aquifer geometry and river-aquifer interaction further downstream of the Limmat valley aquifer. 

In [ ]:
du.display_image(
    image_filename='Detail_outflow.png', 
    image_folder='1_perceptual_model',
    caption='Detail of the outflow boundary condition at piezometer 481. The groundwater level at this piezometer is used as a fixed-head boundary condition for the groundwater flow model. The outflow boundary is placed far enough away from the area of interest, so that the groundwater flow in the area of interest is not influenced by the boundary condition.'
)



We now have an understanding of the geometry of the aquifer. Let's simplify update our perceptual model with the information we have gathered so far: 

- The aquifer does not have a direct hydraulic connection to lake Zurich in the east. 
- It is probably heavily influenced by the river Sihl in the east and the river Limmat in the north.
- Since groundwater flow in river valleys generally follows the topography, we can assume that the groundwater flow in the Limmat valley aquifer is directed from south-east to west. This general flow direction is corroborated by the groundwater flow arrows which become visible when we zoom in on the GIS-browser map (visit the GIS browser to zoom in).
- The aquifer thickness is highly variable. To adequately represent the known geometry of the aquifer, a 3D model with a high spatial resolution is required. We will first opt for a 2D model with a simplified geometry for the case study work to keep the model lean and fast on JupyterHub. 
- The simplified geometry will show a larger thickness in the south-east with a gradual thinning towards the west. The thickness will be represented by a single layer with a variable thickness. We will use the thickness values from the GIS-browser to define the thickness of the aquifer in our model whereby we will have to map the isolines of groundwater thickness to the model grid (see Figure 3).
- The aquifer extends boyond the boundaries of the city of Zurich. Since we are intersted mainly in applying our numerical experiment in the city region, this means that we will have to make assumptions about the outflow boundary of the aquifer.

In [ ]:
du.display_image(
    image_filename='perceptual_model_01_simple_geometry.png', 
    image_folder='1_perceptual_model',
    caption='Figure 3: Illustration of the refined perceptual model, including information about the aquifer shape.'
)

<div style="padding: 1em; margin: 1em 0; border-left: 3px solid #8e44ad; background-color: #f5eef8;">
<strong>🤔 Think about it:</strong><br>
Take a few minutes to explore the available layers in the GIS-browser of the canton of Zurich. What do you think are the most important layers for groundwater modeling? Why?
</div>

In [ ]:
create_section_completion_marker(1)  # Mark section 1 complete

# Climate
Climate data is used to estimate the areal recharge of the aquifer from infiltration of precipitation. It is further used to estimate the lateral inflow from the hills in the north and south of the Limmat valley aquifer. The lateral inflow is estimated using a water balance approach on the hill sides.

## Areal Recharge
We get climate data from the Swiss Federal Office of Meteorology and Climatology (MeteoSwiss). The data is available for download at the following link: [MeteoSwiss](https://www.meteoswiss.admin.ch/services-and-publications/applications/measurement-values-and-measuring-networks.html#param=messnetz-klima&table=false&station=SMA&compare=y&chart=year) and made available in the data directory of the zurich case study repository. [\[4\]](#references) 
The closest station to the Limmat valley aquifer is the Fluntern station. The data is available for the years 1991-2020. Let's have a look at the data. 

In [ ]:
data_path = os.path.abspath(
    os.path.join('case_study_zurich', 'data', 'climate'))
# Test if this is a valid path
if not os.path.exists(data_path):
    print(f"Path {data_path} does not exist.")

climate_norms = cu.read_climate_data(data_path, station_string='Fluntern')

# Inspect the climate norms
display(climate_norms)

> **🤔 Think about it**  
> We are looking at the climate data mostly to get an idea about groundwater recharge. In an aquifer like the Limmat valley aquifer, which is located in a densely populated area, we have to consider that the recharge is not only influenced by the climate but also by human activities. What do you think are the most important human activities that influence groundwater recharge in this area?

Note that we not only have temperature and precipitation data but a wide range of other climate variables which we can use to calculate potential evaporation. However, since the Limmat valley aquifer is located in a built-up area, we will have to make some assumptions about the land use and the resulting potential evaporation. Approximately 80% to 90% of the upper boundary of the aquifer is built-up, with only a small fraction of the area being green space or railway tracks. According to Qui and colleagues, urban evaporation in Swiss climate can amount to about 20% of the precipitation[\[6\]](@references), that would be around 200 mm/year for the Limmat valley aquifer. We will use this value as a first approximation for the potential evaporation in our model. 

Figure 4 below shows the monthly average temperature and precipitation for the Fluntern station. 

In [ ]:
plt, fig = cu.plot_climate_data(climate_norms)
plt.show()

We now have first data concerning the fluxes into and out of the aquifer. We can now update our perceptual model with the information we have gathered so far:
- The average annual precipitation in the Limmat valley aquifer is 1108 mm/year. 
- We assume that the actual evaporation is about 20% of the precipitation, which would be around 200 mm/year (see Figure 5).
- The literature sugests that between 5% and 15% of the precipitation infiltrates into the aquifer [\[7\]](@references). We will use a value of 10% as a first approximation, which would be around 110 mm/year.
- The rest of the precipitation runs off, via the rainwater drainage system, into the river Sihl and the river Limmat. That would be about 70% of the precipitation, which would be around 770 mm/year.

In [ ]:
du.display_image(
    image_filename='perceptual_model_02_atmospheric_fluxes.png', 
    image_folder='1_perceptual_model',
    caption='Figure 5: Adding first estimates of atmospheric forcing to perceptual model.'
)

In [ ]:
create_section_completion_marker(2)  # Mark section 2 complete

## Lateral Inflow from Hills
The lateral inflow from the hills in the north and south of the Limmat valley aquifer is estimated using a water balance approach on the hill sides. We will use the same climate data as for the areal recharge to estimate the lateral inflow.

We can use the same approach as for the areal recharge to estimate the lateral inflow from the hills. Contrary to the areal recharge, we require the area of the hills in the north and south of the Limmat valley aquifer to calculate the lateral inflow. 

As a first approximation, we manually delineate the area of the hills in the north and south of the Limmat valley aquifer using QGIS. We can then use the climate data to estimate the lateral inflow from the hills. 

In [ ]:
du.display_image(
    image_filename='rough_first_manual_catchment_delineation_to_estimate_area_for_lateral_inflow.png', 
    image_folder='1_perceptual_model',
    caption='Figure: Rough first manual catchment delineation to estimate area for lateral inflow. The area is delineated using the height model of the area (here DHM200). The area is used to estimate the lateral inflow from the hills in the north and south of the Limmat valley aquifer. The delineation is not precise and should be refined in a later step. The area relevant for the lateral inflow estimation is 15 km2 in the south and 11 km2 in the north of the Limmat valley aquifer.'
)

Please note, in this very first iteration of the perceptual model, we use crude approximations. A proper catchment delineation would require a more detailed analysis of the topography and the hydrological conditions in the area. You find step-by-step instructions on how to delineate a catchment in many places on the internet, for example here: [https://hydrosolutions.github.io/caham_book/geospatial_data.html#sec-catchment-delineation](https://hydrosolutions.github.io/caham_book/geospatial_data.html#sec-catchment-delineation). 

Because, the hillsides are rather steep and feature forested areas, we assume that the lateral inflow of groundwater into the aquifer is about 20% of the precipitation in the area.  

# River-aquifer interaction

## River Geometry & Recharge
The river-aquifer interaction is an important component of the groundwater flow model. The geometry of the river and its interaction with the aquifer will influence the groundwater flow direction and the groundwater levels in the aquifer.

In practice, a modeler will use measured profiles of the river bed to represent the river geometry in the model. For this course, we do not have access to profiles of the river bed so we have to make some assumptions about the river geometry.

If you are based in Zurich, you can visit the rivers, for example on a bicycle tour. Visiting your model area is a great way to get a feel for the area and to understand the hydrological processes at play. If you don't have the opportunity to visit the area, you can use the GIS-browser or download selected tiles of the high-resolution elevation model to get an idea of the river geometry and ask a local project partner to help you build the perceptual model. 

### Interactive Exploration of River-Aquifer Interaction

The figure below provides an interactive tool to help you visualize the complex relationship between a river and an adjacent unconfined aquifer. It consists of two main parts:

1.  **Left Plot (Physical Cross-Section):** This shows a physical model of the river and the groundwater system. You can see the water level in the river (`H_riv`) and the water table in the aquifer (`H_aq`).
2.  **Right Plot (Flux Relationship):** This is a conceptual graph that shows the calculated flux (flow rate, `Q_riv`) between the river and the aquifer as a function of the aquifer's head. The red dot indicates the current state of the system shown on the left.

**How to Use This Figure:**

Use the two sliders below the plot to change the **Aquifer Head (`H_aq`)** and the **River Stage (`H_riv`)**. As you move the sliders, observe how the system changes in both plots. Pay close attention to the title, the equation, the direction of the flow (indicated by the arrow), and the position of the red dot on the flux curve.

**Scenarios to Explore:**

Try to create the following three fundamental conditions:

1.  **Gaining Stream:**
    *   **How to create it:** Set the aquifer head (`H_aq`) to be *higher* than the river stage (`H_riv`).
    *   **What to observe:** Water flows from the aquifer into the river. The flux (`Q_riv`) is negative (flow *out of* the groundwater). The head difference (`ΔH`) is between `H_aq` and `H_riv`. Notice on the right plot that the red dot is in the negative flux region.

2.  **Losing Stream (Connected):**
    *   **How to create it:** Set the river stage (`H_riv`) to be *higher* than the aquifer head (`H_aq`), but keep `H_aq` *above* the riverbed bottom (`R_bot`, the dashed brown line).
    *   **What to observe:** Water flows from the river into the aquifer. The flux is positive and its magnitude depends directly on the head difference (`ΔH`). As you lower `H_aq`, the flux increases. The red dot on the right plot moves down the sloped part of the curve.

3.  **Losing Stream (Disconnected):**
    *   **How to create it:** Lower the aquifer head (`H_aq`) until it is *below* the riverbed bottom (`R_bot`).
    *   **What to observe:** A yellow "Vadose Zone" appears between the riverbed and the water table, indicating they are no longer directly connected. The flow from the river now seeps downward at a constant, maximum rate. The flux no longer depends on `H_aq`. Notice that the head difference (`ΔH`) for the calculation is now between `H_riv` and `R_bot`. On the right plot, the red dot moves onto the vertical part of the curve, showing that the flux remains constant even if you continue to lower `H_aq`.

In [ ]:
ru.plot_river_aquifer_interaction()

## River Discharge & River Water Levels
The federal office for the environment (FOEN) is the first address for hydrological data in Switzerland. You will find all surface water monitoring sites under [map.geo.admin.ch](https://map.geo.admin.ch) under layer *Hydrological gauaging stations* [\[5\]](#references) (see Figure 2). 

In [ ]:
du.display_image(
    image_filename='BAFU_HydrologicalGaugingStations.png', 
    image_folder='1_perceptual_model',
    caption='Figure 2: Hydrological gauging stations layer available at map.geo.admin.ch.' 
)

When you zoom in to the Limmat valley aquifer (Figure 3), you will find the gauging stations of the rivers Sihl and Limmat. A next gauging station on the river Limmat is located at the city of Baden, downstream of a run-by-the-river hydropower plant, and therefore not relevant for our study. The gauging station on the river Sihl is located at the city of Zurich, upstream of the confluence with the river Limmat. The gauging station on the river Limmat is located at the city of Zurich, downstream of the confluence with the river Sihl. 

In [ ]:
du.display_image(
    image_filename='BAFU_HydrologicalGaugingStations_closeup.png',
    image_folder='1_perceptual_model',
    caption='Figure 3: Locations of hydrological gauging stations near the Limmat valley aquifer.', 
)

A click on the gauaging station will bring you directly to the station site made available by the federal office for the environment (FOEN). The gauging station on the river Sihl is called [*Sihl - Zürich, Sihlhölzli*](https://www.hydrodaten.admin.ch/de/seen-und-fluesse/stationen-und-daten/2176) and has ID 2176 and the gauging station on the river Limmat is called [*Limmat - Zürich, Unterhard*](https://www.hydrodaten.admin.ch/de/seen-und-fluesse/stationen-und-daten/2099) and has ID 2099. The IDs are unique numeric station identifiers and typically required to retrieve data from data repositories or APIs. 

> **🤔 Think about it**  
> For surface water balancing, discharge is the important variable. However, when it comes to flooding or river-aquifer interaction, the water level is the more important variable. Why do you think that is?

From the station sites, we see current water levels, typically over the past 7 days but no water level dynamics. Thankfully, this data can be requested from FOEN. You will find a copy of the data in the data directory of the zurich case study repository. The data is available as a csv file. Please note that the data from the most recent years is provisional data and may be subject to change.

Let's have a look at the data. 


In [ ]:
# Define the path to river data
river_data_path = os.path.abspath(
    os.path.join('case_study_zurich', 'data', 'rivers'))

# Check if path exists
if not os.path.exists(river_data_path):
    print(f"Path {river_data_path} does not exist.")
else:
    try:
        # Plot the typical yearly evolution of river water levels
        fig, axes = ru.plot_yearly_river_levels(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020,
            figsize=(12, 8)
        )
        
        # Display the plot
        plt.show()
        
        # Get summary statistics
        summary = ru.get_river_level_summary(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020
        )
        
        print("\n=== RIVER WATER LEVEL SUMMARY (2010-2020) ===")
        print(f"\nSihl River ({summary['sihl']['station_name']}):")
        print(f"  Mean water level: {summary['sihl']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['sihl']['min']:.3f} - {summary['sihl']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['sihl']['std']:.3f} m")
        print(f"  Total range: {summary['sihl']['range']:.3f} m")
        
        print(f"\nLimmat River ({summary['limmat']['station_name']}):")
        print(f"  Mean water level: {summary['limmat']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['limmat']['min']:.3f} - {summary['limmat']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['limmat']['std']:.3f} m")
        print(f"  Total range: {summary['limmat']['range']:.3f} m")
        
    except ImportError as e:
        print(f"Plotting functionality not available: {e}")
        print("However, we can still analyze the data...")
        
        # Get summary statistics even without plotting
        summary = ru.get_river_level_summary(
            data_path=river_data_path,
            start_year=2010,
            end_year=2020
        )
        
        print("\n=== RIVER WATER LEVEL SUMMARY (2010-2020) ===")
        print(f"\nSihl River ({summary['sihl']['station_name']}):")
        print(f"  Mean water level: {summary['sihl']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['sihl']['min']:.3f} - {summary['sihl']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['sihl']['std']:.3f} m")
        print(f"  Total range: {summary['sihl']['range']:.3f} m")
        
        print(f"\nLimmat River ({summary['limmat']['station_name']}):")
        print(f"  Mean water level: {summary['limmat']['mean']:.3f} m a.s.l.")
        print(f"  Range: {summary['limmat']['min']:.3f} - {summary['limmat']['max']:.3f} m a.s.l.")
        print(f"  Standard deviation: {summary['limmat']['std']:.3f} m")
        print(f"  Total range: {summary['limmat']['range']:.3f} m")

No additional river gauging stations are maintained by the cantonal office for waste, water, energy and air (Amt für Abfall, Wasser, Energie und Luft (AWEL)) [\[6\]](#references). 

Let's bring the river water levels in context with the groundwater levels near the gauging stations. 

In [ ]:
# Sihl River Data
sihl_gw_mean = 400.8
sihl_gw_high = 403.5
sihl_river_mean = 412.347
sihl_river_high = 413.439
sihl_depth_mean = 0.3

# Limmat River Data
limmat_gw_mean = 399.5
limmat_gw_high = 400.5
limmat_river_mean = 400.285
limmat_river_high = 401.822
limmat_depth_mean = 0.7

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 14))
fig.suptitle("River and Groundwater Level Cross-Sections", fontsize=16, y=0.95)

# Plot Sihl
ru.plot_cross_section(ax1, "River Sihl", sihl_gw_mean, sihl_gw_high, sihl_river_mean, sihl_river_high, sihl_depth_mean)

# Plot Limmat
ru.plot_cross_section(ax2, "River Limmat", limmat_gw_mean, limmat_gw_high, limmat_river_mean, limmat_river_high, limmat_depth_mean)

plt.tight_layout(rect=[0, 0, 1, 0.93]) # Adjust layout to make space for suptitle
plt.show()

From the two conceptualized cross-section at the gauge location on the river Sihl we see that we are dealing with a loosing stream which is disconnected from the groundwater table. We therefore have a constant flux from the river into the aquifer. The flux is not influenced by the groundwater table in the aquifer. The flux is only influenced by the river stage and the river bed elevation.
The river Limmat is also a loosing stream at the location of the gauge. However, from the conceptualized cross-section we see that the river Limmat is very likely connected to the groundwater table. We have to account for a capillary fringe of up to 30 cm (depending on the pore size distribution of the soil matrix). The flux from the river into the aquifer is influenced by the groundwater table in the aquifer. The flux is further influenced by the river stage and the river bed elevation.

To simplify the first rough estimation of groundwater recharge from river infiltration, we assume disconnected streams for both rivers and an average river water level of 0.5 m for both rivers and a hydraulic conductivity of the Schmutzschicht of $10^{-6} - 10^{-7}$ m/s. This is to give us a very rough first estimate of the order of magnitude of the river-aquifer interaction.

$$q_{riv} = K_{Schmutzschicht} \cdot (H_{riv} - H_{aq}) \approx 5 \cdot 10^{-7} - 5 \cdot 10^{-8} m/s $$

Where:
- $q_{riv}$ is the flux from the river into the aquifer (m/s)
- $K_{Schmutzschicht}$ is the hydraulic conductivity of the Schmutzschicht (m/s)
- $H_{riv}$ is the river water level (m)
- $H_{aq}$ is the groundwater level in the aquifer (m)

The area of the rivers Sihl and Limmat we can estimate from the geopackage layer "Oeffentliche Oberflaechengewaesser" provided by the canton of Zurich ([https://opendata.swiss/en/dataset/offentliche-oberflachengewasser](https://opendata.swiss/en/dataset/offentliche-oberflachengewasser), accessed 2025-07-06) using QGIS. 

In [ ]:
# Download the shapes of the rivers and the locations of the gauging stations. 
gauges_file_path = download_named_file(
    name='gauges',
    data_type='gis'
)
rivers_file_path = download_named_file(
    name='rivers',
    data_type='gis'
)
from map_utils import plot_model_delineation_map
plot_model_delineation_map(
    gw_depth_path=gw_map_path, 
    rivers_path=rivers_file_path,
    gauges_path=gauges_file_path,
)

In [ ]:
create_section_completion_marker(3)  

## Monitoring the Limmat Valley Aquifer
To get a first idea about the groundwater levels in the Limmat valley aquifer, we will have a look at the groundwater map.

Several authorities do groundwater monitoring in the Limmat valley aquifer. We will start with the federal office for the environment (FOEN) which maintains a network of groundwater observation wells. You find an overview over the available groundwater monitoring sites at [https://map.geo.admin.ch/](https://map.geo.admin.ch/) in layer *Groundwater level/spring discharge* [\[7\]](#references). One monitoring well maintained by FOEN is located in the Limmat valley aquifer but far downstream of our area of interest in the city center. Further, this well is a drinking water production well and can therefore not be used as an outflow boundary.  

Few groundwater observation wells are operated by the cantonal office of the environment (AWEL) (Figure 4).



In [ ]:
du.display_image(
    image_filename='GWmonitoring_locations_AWEL.png', 
    image_folder='1_perceptual_model',
    caption='Figure 4: Monitoring wells in the Limmat valley aquifer. Yearbook sheets for each site can be accessed through the popup window from each site. Source: https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/grundwasserstaende.html, accessed 2025-05-01.'
)

> **🤔 Think about it**  
> What are the major hydrological processes in the Limmat valley aquifer?  
> Where would you set the boundaries of the Limmat valley aquifer?

In [ ]:
create_section_completion_marker(5)  

# TODO: Finished perceptual model showing all fluxes

## References
[\[1\]](#the-limmat-valley-aquifer) Hug J., and Beilick, A. (1934): Die Grundwasserverhältnisse des Kantons Zürich. In: Beiträge zur Geologie der Schweiz - Geotechnische Serie - Hydrologie. German. Available online here: https://scnat.ch/de/uuid/i/0bd7aa3b-0bd7-5d54-9e2f-597a42dada50-Die_Grundwasserverh%C3%A4ltnisse_des_Kantons_Z%C3%BCrich (accessed 2025-05-01)   
[\[2\]](#the-limmat-valley-aquifer) Doppler, T., Hendricks Franssen, H.-J., Kaiser H.-P., Kuhlman U., Stauffer, F. (2007): Field evidence of a dynamic leakage coefficient for modelling river–aquifer interactions. Journal of Hydrology, Volume 347, Issues 1–2, DOI: https://doi.org/10.1016/j.jhydrol.2007.09.017.  
[\[3\]](#the-limmat-valley-aquifer) GIS-browser of the canton of Zurich: https://www.gis.zh.ch/ (accessed 2025-05-01)  
[\[4\]](#the-limmat-valley-aquifer) Freeze, R. A., and Cherry, J. A. (1979): Groundwater. Prentice-Hall, Englewood Cliffs, New Jersey, USA. ISBN: 978-0133653122. Open-book available here: [https://gw-project.org/books/groundwater/](https://gw-project.org/books/groundwater/)   
[\[4\]](#climate) MeteoSwiss: https://www.meteoswiss.admin.ch/services-and-publications/applications/measurement-values-and-measuring-networks.html#param=messnetz-klima&table=false&station=SMA&compare=y&chart=year (accessed 2025-05-01)  
[\[5\]](#climate) Burt, C. M., Mutziger, A. J., Allen, R. G., and Howell, T. A. (2005): Evaporation Research: Review and Interpretation. Journal of Irrigation and Drainage Engineering, 131(1), 1–13. DOI: [https://doi.org/10.1061/(ASCE)0733-9437(2005)131:1(1)](https://doi.org/10.1061/(ASCE)0733-9437(2005)131:1(1)).  
[\[6\]](#climate) Qui, Guo Yu, Yan, Chunhua, Liu, Yuanbo (2023): Urban evapotranspiration and its effects on water budget and energy balance: Review and perspectives. Earth-Science Reviews, Volume 246. DOI: [https://doi.org/10.1016/j.earscirev.2023.104577](https://doi.org/10.1016/j.earscirev.2023.104577)  
[\[7\]](#climate) Lerner, David N. (2002): Identifying and quantifying urban recharge: a review. Hydrogeology Journal, Volume 10, Issue 1, pp 143–152. DOI: [https://doi.org/10.1007/s10040-001-0177-7](https://doi.org/10.1007/s10040-001-0177-1)   
[\[5\]](#river-discharge) Locations of hydrological gauging stations maintained by the Federal Office for the Environment (FOEN): https://map.geo.admin.ch (accessed 2025-05-01)
[\[6\]](#river-discharge) Locations of hydrological gauging stations maintained by the cantonal office for waste, water, energy, and air (AWEL): https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/abfluss-wasserstand.html (accessed 2025-05-01)  
[\[7\]](#monitoring-the-limmat-valley-aquifer) Locations of groundwater monitoring wells maintained by Federal Office for the Environment (FOEN): https://www.zh.ch/de/umwelt-und-natur/wasser/grundwasser/monitoring.html (accessed 2025-05-01)  
[\[8\]](#monitoring-the-limmat-valley-aquifer) Cantonal office of the environment (AWEL): https://www.zh.ch/de/umwelt-tiere/wasser-gewaesser/messdaten/grundwasserstaende.html (accessed 2025-05-01)
